In [57]:
%matplotlib widget
import matplotlib.pyplot as plt
import matplotlib
from scipy.linalg import norm, eig
from scipy.optimize import minimize
import numpy as np

In [3]:
sx = 1/np.sqrt(2)*np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
sx2 = 1/2.*np.array([[0, 1], [1, 0]])
sy = 1/np.sqrt(2)*np.array([[0, -complex(0, 1), 0],
                            [complex(0, 1), 0, -complex(0, 1)], [0, complex(0, 1), 0]])
sy2 = 1/2.*np.array([[0, -complex(0, 1)], [complex(0, 1), 0]])
sz2 = 1/2.*np.array([[1, 0], [0, -1]])
sz = np.array([[1, 0, 0], [0, 0, 0], [0, 0, -1]])
id3 = np.identity(3)
id2 = np.identity(2)

M = sx+sy

In [4]:
def hamiltonian_full(bx, by, bz):
    hzfs = d * (np.linalg.matrix_power(sz, 2) - 2/3.*id3) + e * \
        (np.linalg.matrix_power(sx, 2) - np.linalg.matrix_power(sy, 2))
    h_ele_zeeman = 2.8*(bx*sx + by*sy + bz*sz)
    h = hzfs+h_ele_zeeman
    #h = hzfs + h_ele_zeeman
    return h

In [5]:
def lorentzian(t, pos, ampl, width):
    return ampl*width**2/((t-pos)**2+width**2)

In [30]:
# diagonalize and find transitions
def find_transitions(b):
    eva, eve = eig(hamiltonian_full(*b))
    #eva = eva.real
    eva = eva-min(eva)
    trans = np.zeros(np.shape(M))
    for i, ev in enumerate(eve):
        for j, ev2 in enumerate(eve):
            trans[i][j] = abs(
                np.dot(np.asmatrix(ev).H.T, np.dot(M, np.asmatrix(ev2).T)))

    # for i, t1 in enumerate(trans):
    #    for j, t2 in enumerate(t1):
    #        if t2 < 0.1:
    #            trans[i][j] = 0
    tr = []
    for i, tr1 in enumerate(trans):
        for j, tr2 in enumerate(tr1):
            if tr2 > 0.5:
                tr.append((tr2, abs(eva[i]-eva[j])))

    tr = np.unique(tr, axis=0)
    t = tr[tr > 1]
    return max(t)

In [7]:
# transform vector from lab system into the four NV systems:
# we just need to find axial and transversal components,
# since transversal directions are equal in hamiltonian, and we assume RF polarization to be diagonally polarized
def lab2nvs(b):
    # the four NV axes in the lab system:
    nv_axes = 1/np.sqrt(3) * np.array([
        [1, 1, 1],
        [1, -1, -1],
        [-1, -1, 1],
        [-1, 1, -1]
    ])
    # projection onto axes:
    axial = np.dot(nv_axes, b)
    transversal = np.sqrt(np.abs(norm(b)**2 - axial**2))
    # let's just assign the transversal component to the x axes
    b_nvs = np.vstack([transversal, np.zeros(4), axial])
    return b_nvs

In [108]:
def find_magnetic_field(fr, bf):
    bdir = bf/norm(bf)
    def minfun(bnorm): return np.abs(fr-find_transitions(bnorm*bdir))
    res = minimize(minfun, 165)
    return float(res.x)

In [25]:
# parameters: MHz, Gauss
e = 0
d = 2880.
a = -3.05
n_z = -4.316e-4
fr = 3220
bdir = np.array([1, 0, 0])
bdir = bdir/norm(bdir)
b = 42.8 * bdir   # b vector in lab system, in Gauss
print(b)

[42.8  0.   0. ]


In [32]:
# solve for all four NV directions
b_nvs = lab2nvs(b)
tr = []
for i in range(np.shape(b_nvs)[1]):
    t = find_transitions(b_nvs[:, i])
    tr.append(t)

tr_all = np.vstack(tr)
print(tr_all)

[[2954.15353169]
 [2954.15353169]
 [2954.15353169]
 [2954.15353169]]


In [27]:
t1 = np.linspace(min(tr_all[:, 0])-10, max(tr_all[:, 0])+10, 10000)
# t1 = linspace(2700, 'max(tr_all[:,1])+10, 10000)
plt.close('all')
matplotlib.interactive(False)
plt.figure()
col = ['r', 'g', 'b', 'y']
for i, t in enumerate(tr_all):
    for l in t:
        plt.plot(t1, lorentzian(t1, l, -l/t.shape[0], 1), col[i])
plt.xlim((min(t1), max(t1)))
plt.ylim((plt.ylim()[0], 0))
plt.xlabel('Frequency [MHz]')
#plt.title('B = %s' % str(b))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
factor_coils = 1  # how much stronger / weaker the field is compared to what we think
# the angle between the field we think we apply and the field we do apply
angle_plane = np.pi/2
angle_z = np.pi/2  # the angle between the field we ...
b_dir_assumed = [np.cos(angle_plane)*np.sin(angle_z),
                 np.sin(angle_plane)*np.sin(angle_z), np.cos(angle_z)]

In [ ]:
def get_lines(phi, theta, factor, fr):
    